In [ ]:
import functions as func
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Preprocessing

In [ ]:
# Um diretorio que contém todos os datasets
path = 'data'

df_dict = func.readCSV_DATA(path)

In [ ]:
df_train = df_dict['train.csv']

In [ ]:
question_lst = []
answer_lst = []

for column in df_train.columns[11:]:
    match = re.match('([a-z]*)_', column)
    if match.group(1) == 'question':
        question_lst.append(column)
    else:
        answer_lst.append(column)

In [ ]:
df_train = func.prepareData(df_train)

# Extracting simple text features

In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
%%time
question_vectorizer = TfidfVectorizer(tokenizer=word_tokenize,
                                      strip_accents='ascii',
                                      stop_words='english',
                                      min_df=3,
                                      max_df=int(df_train.shape[0]/30))

question_X = question_vectorizer.fit_transform(df_train['question_text_clean'])

In [ ]:
%%time
answer_vectorizer = TfidfVectorizer(tokenizer=word_tokenize,
                                    strip_accents='ascii',
                                    stop_words='english',
                                    min_df=3,
                                    max_df=int(df_train.shape[0]/30))
answer_X = answer_vectorizer.fit_transform(df_train['answer_text_clean'])

In [ ]:
question_y = df_train[question_lst]
answer_y = df_train[answer_lst]

In [ ]:
X_train_question, X_test_question, y_train_question, y_test_question = train_test_split(
    question_X, question_y, test_size=0.2)

X_train_answer, X_test_answer, y_train_answer, y_test_answer = train_test_split(
    answer_X, answer_y, test_size=0.2)

# Train a Linear Regression in NN format

## Create model - Answer

In [ ]:
output_dim = len(answer_lst)
input_dim = len(answer_vectorizer.vocabulary_)

answer_model = func.create_model(output_dim, input_dim, 'binary_crossentropy')

answer_model = func.train_model(answer_model, X_train_answer, y_train_answer, X_test_answer,
                                y_test_answer, batch_size=128, nb_epoch=60, verbose=0)

In [ ]:
func.spearman_corr(y_train_answer, answer_model.predict(X_train_answer.toarray()))

In [ ]:
func.spearman_corr(y_test_answer, answer_model.predict(X_test_answer.toarray()))

## Create model - Question

In [ ]:
output_dim = len(question_lst)
input_dim  = len(question_vectorizer.vocabulary_)

question_model = func.create_model(output_dim, input_dim, 'binary_crossentropy')

In [ ]:
question_model = func.train_model(question_model, X_train_question, y_train_question, X_test_question,
                                  y_test_question, batch_size=64, nb_epoch=100, verbose=1)

In [ ]:
func.spearman_corr(y_test_question, question_model.predict(X_test_question))

# Submission

In [ ]:
sub = df_dict['sample_submission.csv']
df_test = df_dict['test.csv']

In [ ]:
df_test = func.prepareData(df_test)

answer_X_test = answer_vectorizer.transform(df_test['answer_text_clean'])
question_X_test = question_vectorizer.transform(df_test['question_text_clean'])

In [ ]:
answer_y_hat_test = answer_model.predict(answer_X_test.toarray())
question_y_hat_test = question_model.predict(question_X_test.toarray())

In [ ]:
y_hat_test = np.hstack((question_y_hat_test, answer_y_hat_test))

In [ ]:
for col_index, col in enumerate(question_lst + answer_lst):
    sub[col] = y_hat_test[:, col_index]

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index = False)